In [1]:
import utils_nlp
import pickle
# dataset_filepaths, dataset_brat_folders = utils.get_valid_dataset_filepaths(parameters)
# dataset = ds.DatasetP(verbose=False, debug=False)
token_to_vector = utils_nlp.load_pretrained_token_embeddings('../../../ML_EntityData/embedding/en/glove.6B.100d.txt')

In [2]:
from vocabulary import Vocabulary

In [3]:
vocab = Vocabulary(token_to_vector,100)

In [4]:
token_indices, character_indices_padded, token_lengths, pattern ,label_indices , label_vector= vocab.parse_conll('../../../ML_EntityData/data/en/train.txt')

In [7]:
label_vector

[array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 

In [7]:
len(vocab.get_embedding(token_to_vector,100))

18416

In [9]:
len(token_to_vector)

400000

In [5]:
vocab.transform_text('my name is Ngoc Linh')

([[1051, 3228, 446, 8337, 8338, 11]],
 [[[40, 43, 0, 0],
   [31, 34, 40, 28],
   [33, 29, 0, 0],
   [16, 46, 37, 44],
   [8, 33, 31, 39],
   [19, 0, 0, 0]]],
 [[2, 4, 2, 4, 4, 1]],
 [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [12]:
import sys
import pickle

In [8]:
pickle.dump(vocab.get_embedding(token_to_vector,100),open('../../vocab.pickle','wb'))

In [11]:
sys.getsizeof(token_to_vector)

20971616